# Notes
This notebook is a playground to help us write code in the main folders. This notebook aims at finding the area for each property, which is something that the data from domain didn't cover. We notice that the area is a very useful feature in determining the rental price, yet due to the time limits, we couldn't integrate this in our pipeline 

We hope to finish this in the near future as this is included in our future plan proposal

In [1]:
import pandas as pd
import pandas as pd
import pandas as pd
import numpy as np
import geopandas as gpd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import glob
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import  DoubleType
import  re
from simpledbf import Dbf5

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "10g") 
    .config("spark.driver.memory", "10g") 
    .config("spark.executor.extraJavaOptions", "-XX:MaxGCMinorPauseMillis=500")
    .getOrCreate()
)

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


your 131072x1 screen size is bogus. expect trouble
23/09/20 14:56:33 WARN Utils: Your hostname, clarissaslaptop resolves to a loopback address: 127.0.1.1; using 172.19.129.177 instead (on interface eth0)
23/09/20 14:56:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/20 14:56:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# this data contains the property polygons for properties in victoria, along with their property f identifier which we will use to link to the address information
sf = gpd.read_file("../data/raw/property_polygon_p_detail/ll_gda2020/esrishape/whole_of_dataset/victoria/VMPROP/V_PROPERTY_MP.shp", engine='pyogrio')[['PROP_PFI', 'geometry']]
# using the simple cylindrical equal area projection for a more accurate claculation of area




In [ ]:
sf.crs = 'epsg:6933'
geod = sf.crs.get_geod()

In [ ]:
# computing area 
sf["area"] = sf.geometry.apply(lambda g: abs(geod.geometry_area_perimeter(g)[0]))
sf = sf.drop(['geometry'], axis=1)
# saving for later use 
sf.to_csv("../data/landing/property_land_area.csv")

In [2]:
# this file contains the address data for all properties in victoria, along with pfi. we read the dbf file because the shapefile data is not needed and too large
dbf = Dbf5('../data/raw/address/ll_gda2020/esrishape/whole_of_dataset/victoria/VMADD/ADDRESS_MAIN.dbf', codec='utf-8')
dbf.to_csv('../data/landing/addresses.csv')

In [5]:
# joining area data with address
addresses = spark.read.csv('../data/landing/addresses.csv', header=True)[['PR_PFI','FLOOR_NO_1', 'BUNIT_ID1', 'HSE_NUM1', 'ROAD_NAME', 'ROAD_TYPE', 'LOCALITY', 'STATE', 'POSTCODE']]
land_area = spark.read.csv("../data/landing/property_land_area.csv", header=True)[['PROP_PFI', 'area']]
addresses = addresses.withColumnRenamed('FLOOR_NO_1', 'APT_LVL')
addresses = addresses.withColumnRenamed('BUNIT_ID1', 'APT_NUM')
addresses = addresses.withColumnRenamed('HSE_NUM1', 'BLDG_NUM')
land_area = land_area.withColumnRenamed('PROP_PFI', 'PR_PFI')
land_area = land_area.drop_duplicates()
addresses = addresses.drop_duplicates(['APT_NUM', 'BLDG_NUM', 'ROAD_NAME', 'ROAD_TYPE', 'LOCALITY','POSTCODE'])

# ensuring all strings are lower case
addresses=addresses.withColumn('ROAD_NAME', F.lower(F.col('ROAD_NAME')))
addresses=addresses.withColumn('ROAD_TYPE', F.lower(F.col('ROAD_TYPE')))
addresses=addresses.withColumn('LOCALITY', F.lower(F.col('LOCALITY')))

# casting apartment number as int because we will be using them to be estimating missing apartment levels 
addresses=addresses.withColumn('APT_LVL', F.col('APT_LVL').cast('int'))
addresses=addresses.withColumn('APT_NUM', F.col('APT_NUM').cast('int'))

# joining datasets 
property_area = addresses.join(land_area, how = 'inner', on = 'PR_PFI')

# separating into houses and apartments so we can calculate apt level for apartments only 
apts = property_area.filter(F.col('APT_NUM').isNotNull() == True)
houses = property_area.filter(F.col('APT_NUM').isNull() == True)
houses = houses.withColumn('num_apt_on_floor', F.lit(None))
houses = houses.withColumn('apt_area', F.lit(None))

# when apartments are higher than 1 storey, they generally have the level = first number 
apts=apts.withColumn('APT_LVL', F.when(F.col('APT_LVL').isNotNull() == True, F.col('APT_LVL'))\
                                        .otherwise(F.floor(F.col('APT_NUM')/100)))

# finding the total number of apartments on a floor of a building
level_grouped = apts.groupby(['APT_LVL',  'BLDG_NUM', 'ROAD_NAME', 'ROAD_TYPE', 'LOCALITY', 'STATE', 'POSTCODE'])\
        .agg(F.median('area').alias('floor_area'),
             F.count('area').alias("num_apt_on_floor"))
# approximating apartment area by dividing land area by number of apartments on that floor 
level_grouped = level_grouped.withColumn("apt_area", F.col('floor_area')/F.col('num_apt_on_floor'))
# joining to apartment data 
apts = apts.join(level_grouped, how='left', on = ['APT_LVL',  'BLDG_NUM', 'ROAD_NAME', 'ROAD_TYPE', 'LOCALITY', 'STATE', 'POSTCODE'])
apts = apts.drop('floor_area')#, 'area')
# combining apartment and house data 
all_properties = apts.unionByName(houses)

In [6]:
apts.limit(20)

APT_LVL,BLDG_NUM,ROAD_NAME,ROAD_TYPE,LOCALITY,STATE,POSTCODE,PR_PFI,APT_NUM,area,num_apt_on_floor,apt_area
0,1241,doveton,street,invermay park,VIC,3350,100030214,2,349.1899076993577,2,164.32998860476073
0,1241,doveton,street,invermay park,VIC,3350,100030206,1,308.1300467196852,2,164.32998860476073
0,17,mouritz,street,warragul,VIC,3820,100013663,1,273.9385187383741,2,136.05079801561078
0,17,mouritz,street,warragul,VIC,3820,100013671,2,270.264673324069,2,136.05079801561078
0,21,pallett,street,golden square,VIC,3555,100073457,1,489.58699106797576,2,1227.4609977206856
0,25,arblaster,street,california gully,VIC,3556,100073954,1,273.48930666036904,4,68.6726030560676
0,25,arblaster,street,california gully,VIC,3556,100073970,4,191.44723909022287,4,68.6726030560676
0,4,moylan,street,bentleigh east,VIC,3165,10021,2,364.4177685556933,2,181.08997783786617
0,5,cook,place,west wodonga,VIC,3690,100084244,1,243.97727836027843,12,19.419149878519722
0,9,saunders,street,west bendigo,VIC,3550,100004610,1,422.2981316377409,4,101.12733876961283


In [8]:
all_properties.filter((F.col('ROAD_NAME') == 'therry')&\
                       (F.col('BLDG_NUM') == '101')).limit(20)

APT_LVL,BLDG_NUM,ROAD_NAME,ROAD_TYPE,LOCALITY,STATE,POSTCODE,PR_PFI,APT_NUM,area,num_apt_on_floor,apt_area
22,101,therry,street,melbourne,VIC,3000,454279239,2215,3160.824837445194,15,210.7216558296796
22,101,therry,street,melbourne,VIC,3000,454279233,2213,3160.824837445194,15,210.7216558296796
38,101,therry,street,melbourne,VIC,3000,454280184,3801,3160.824837445194,14,225.77320267465672
24,101,therry,street,melbourne,VIC,3000,454279993,2406,3160.824837445194,15,210.7216558296796
24,101,therry,street,melbourne,VIC,3000,454279999,2414,3160.824837445194,15,210.7216558296796
37,101,therry,street,melbourne,VIC,3000,454279641,3706,3160.824837445194,14,225.77320267465672
16,101,therry,street,melbourne,VIC,3000,454279056,1612,3160.824837445194,15,210.7216558296796
35,101,therry,street,melbourne,VIC,3000,454279581,3503,3160.824837445194,14,225.77320267465672
7,101,therry,street,melbourne,VIC,3000,454280739,713,3160.824837445194,16,197.55155234032463
34,101,therry,street,melbourne,VIC,3000,454279560,3407,3160.824837445194,14,225.77320267465672


In [3]:
all_properties.repartition("POSTCODE")\
        .write\
        .option("header",True) \
        .partitionBy("POSTCODE") \
        .mode("overwrite") \
        .csv("../data/landing/area_by_locality")

In [3]:
# this is the data we scraped from the domain website
domain_data = pd.read_csv("../data/curated/real_estate_2023.csv")[['id', "address"]]


# the name column includes address which we will try to extract street number, street name, street type, lcoality and postcode from later on 
domain_data = domain_data[domain_data['address'].isna() == False]
domain_data['address'] = domain_data['address'].str.lower()
domain_data['address'] = domain_data['address'].str.replace(",", " ")
domain_data['address'] = domain_data['address'].str.replace(".", " ")
domain_data['address'] = domain_data['address'].str.replace("'", "")

# dictionary that returns the proper road type when given the abbreviated version 
road_type_dict = {'':None}
# retrieving road types from the addresses data and adding to dictionary
road_types = addresses.filter(F.col('ROAD_TYPE').isNotNull() == True).select('ROAD_TYPE').distinct().toPandas()['ROAD_TYPE'].to_list()
for i in road_types:
    road_type_dict[i] = i

# list of most common abbreviations, and their corresponding proper names 
abbreviations = ['st',  'av', 'ave', 'rd', 'hwy', 
                 'pd', 'pde', 'blvd', 'bvd', 
                 'ln', 'gr', 'dr', 'crs', 'cres', 'cr',
                 'cl', 'ct', 'crt', 'pl', 'esp', 'espl', 'cct' ]
full_road_type = ['street',  'avenue', 'avenue', 'road', 'highway',
                 'parade', 'parade', 'boulevard', 'boulevard',
                 'lane', 'grove', 'drive',  'crescent', 'crescent', 'crescent',
                 'close', 'court', 'court', 'place', 'esplanade', 'esplanade', 'circuit']
for i in range(len(abbreviations)):
    road_type_dict[abbreviations[i]] = full_road_type[i]

# list of possible terms that property road types are listed as  
road_types = road_types + abbreviations  

# extracting the street number, postcodes and the words between
num_words_postcode = domain_data['address'].str.extract(r'(\d+\-?[ab]?\/?\d*\-?\d*) (\D*) (?:vic)? (3\d{3})')
num_words_postcode = num_words_postcode.rename(columns={0:'number', 1:'street_suburb', 2:'POSTCODE'}).reset_index(drop=True)

# initialising locality
num_words_postcode['LOCALITY'] = pd.Series(None, range(len(num_words_postcode)))

# list of all localities in victoria, extracted from address data, with the length of names calculated 
localities = addresses.filter(F.col('LOCALITY').isNotNull() == True).select('LOCALITY').distinct().toPandas()
localities['name_len'] = localities['LOCALITY'].str.len()

# sorting so that locality names which include other locality names (eg. north warrandyte vs warrandyte) are identified first 
localities = localities.sort_values(by = 'name_len', ascending=False).reset_index(drop=True)

# for entries without a locality identified, try to see if it is located in name 
# we check whether the last letters equal the locality name because some locality names are also road names (eg. blackburn vs blackburn rd)
for i in range(len(localities)):
    name_len = localities.loc[i,'name_len']
    locality = localities.loc[i,'LOCALITY']
    num_words_postcode['LOCALITY'] = num_words_postcode['LOCALITY'].mask((num_words_postcode['LOCALITY'].isna() == True) & 
                                                         ((num_words_postcode['street_suburb'].str[-name_len:]) == locality), locality)
# joining locality to domain data so we can drop invalid entries 
domain_data = pd.concat([domain_data,num_words_postcode], axis = 1)
domain_data = domain_data[domain_data['LOCALITY'].isna() == False].reset_index(drop=True)

#calculating locality name length which we will be using again. instead of joining because this is likely faster 
domain_data['name_len'] = domain_data['LOCALITY'].str.len()

# street name and type contained in this column
domain_data['street'] = pd.Series([x[:-y] for x, y in zip(domain_data['street_suburb'], domain_data['name_len'])])

# finding road type by checking if valid road types are contained in the string 
domain_data['ROAD_TYPE'] = pd.Series(None, range(len(domain_data)))
for i in road_types:
    domain_data['ROAD_TYPE'] = domain_data['ROAD_TYPE'].mask(((domain_data['ROAD_TYPE'].isna() == True) & 
                                                         (domain_data['street'].str.contains(i))), i)
domain_data = domain_data[domain_data['ROAD_TYPE'].isna() == False].reset_index(drop=True)

# finding road name (the test before road type)
domain_data['ROAD_NAME'] = pd.Series([re.findall('(\D*) ' + str(y), x) for x, y in zip(domain_data['street'], domain_data['ROAD_TYPE'])])
domain_data['ROAD_NAME'] = domain_data['ROAD_NAME'].str[0]
domain_data = domain_data[domain_data['ROAD_NAME'].isna() == False].reset_index(drop=True)

# replacing abbreviated road types with proper term 
domain_data['ROAD_TYPE'] = domain_data['ROAD_TYPE'].apply(lambda x: road_type_dict[x])

# finding apartment and building number (since the format is generally apt_number/building_number)
domain_data['number'] = domain_data['number'].str.split('/')
domain_data['BLDG_NUM'] = domain_data['number'].str[-1].str.extract(r'(\d+)')
domain_data['APT_NUM'] = np.where(domain_data['number'].str.len() == 2, domain_data['number'].str[0], None)
domain_data = domain_data[domain_data['BLDG_NUM'].isna() == False].reset_index(drop=True)

In [4]:
#getting list of postcodes in domain listings. we will iterate over these when joining domain to area data as it is too expensive to match domain data to 3 mill+ addresses
postcodes = domain_data['POSTCODE'].unique()

In [5]:
# saving by postcode
domain_data = spark.createDataFrame(domain_data[['id', 'address', 'APT_NUM', 'BLDG_NUM', 'ROAD_NAME', 'ROAD_TYPE', 'LOCALITY', 'POSTCODE']])\
        .drop_duplicates(['APT_NUM','BLDG_NUM','ROAD_NAME','ROAD_TYPE','LOCALITY','POSTCODE'])
domain_data.repartition("POSTCODE")\
        .write\
        .option("header",True) \
        .partitionBy("POSTCODE") \
        .mode("overwrite") \
        .csv("../data/landing/domain_data_by_postcode")

In [6]:
joined = []
# for each postcode join domain data to area data by address 
for postcode in postcodes:
    area_path = glob.glob(f'../data/landing/area_by_postcode/POSTCODE={postcode}/*.csv')[0]
    temp_area = spark.read.csv(area_path, header=True)
    domain_data_path = glob.glob(f'../data/landing/domain_data_by_postcode/POSTCODE={postcode}/*.csv')[0]
    temp_domain_data = spark.read.csv(domain_data_path, header=True)

    temp_domain_data = temp_domain_data.withColumn('POSTCODE', F.lit(postcode))

    # joining data. the readon we did not join on a list of columns is because null values apt number will be considered not a match, when we want them to be (houses have null-values apt numbers )
    cond = (((temp_domain_data['APT_NUM'] == temp_area['APT_NUM']) | 
        ((temp_domain_data['APT_NUM'].isNull() == True) & (temp_area['APT_NUM'].isNull() == True))) &\
        (temp_domain_data['BLDG_NUM'] == temp_area['BLDG_NUM']) &\
        (temp_domain_data['ROAD_NAME'] == temp_area['ROAD_NAME']) &\
        (temp_domain_data['ROAD_TYPE'] == temp_area['ROAD_TYPE']))
    # joining data 
    joined.append(temp_domain_data.join(temp_area, how = 'left', on = cond).drop(temp_area['APT_NUM'],temp_area['BLDG_NUM'],temp_area['ROAD_NAME'],temp_area['ROAD_TYPE'],temp_area['LOCALITY']))

In [8]:
# writing into csv as it is too computationally expensive to union all dataframes 
matched = []
unmatched = []
for i in joined:
    matched.append(i.filter(F.col('area').isNotNull() == True))
    unmatched.append(i.filter(F.col('area').isNull() == True))
n = 0
for i in matched:
    i.coalesce(1).write.save(path='../data/curated/matched_csv.csv', format='csv', mode='append', sep=',')
    n = n + 1

n = 0
for i in unmatched:
    i.coalesce(1).write.save(path='../data/curated/unmatched_csv.csv', format='csv', mode='append', sep=',')
    n = n + 1

In [9]:
# schema of matched/unmatched data 
columns = ['APT_NUM','BLDG_NUM','ROAD_NAME','ROAD_TYPE','id','address','st_number','SUBURB','POSTCODE','APT_LVL','LOCALITY','STATE','PR_PFI','area']
schema = StructType([StructField('id', StringType(), True),\
                    StructField('address', StringType(), True),\
                    StructField('APT_NUM', IntegerType(), True),\
                    StructField('BLDG_NUM', StringType(), True),\
                    StructField('ROAD_NAME', StringType(), True),\
                    StructField('ROAD_TYPE', StringType(), True),\
                    StructField('LOCALITY', StringType(), True),\
                    StructField('POSTCODE', StringType(), True),\
                    StructField('APT_LVL', StringType(), True),\
                    StructField('STATE', StringType(), True),\
                    StructField('PR_PFI', StringType(), True),\
                    StructField('area', DoubleType(), True)])

all_matched = spark.read.csv('../data/curated/matched_csv.csv', schema=schema)
all_unmatched = spark.read.csv('../data/curated/unmatched_csv.csv', schema=schema).drop("STATE", "PR_PFI", "area")

In [15]:
all_matched.limit(20)

id,address,APT_NUM,BLDG_NUM,ROAD_NAME,ROAD_TYPE,LOCALITY,POSTCODE,APT_LVL,STATE,PR_PFI,area
15,32/18 queen stree...,32,18,queen,street,melbourne,3000,0,VIC,52519796,12.05167394131422
493,1/24-38 little bo...,1,24,little bourke,street,melbourne,3000,0,VIC,52788949,33.43753358763756
59,2/6 anthony stree...,2,6,anthony,street,melbourne,3000,0,VIC,50078519,35.45786546121041
524,l44/81 abeckett s...,44,81,abeckett,street,melbourne,3000,0,VIC,426603630,11.863994440324765
538,7/9-13 anthony st...,7,9,anthony,street,melbourne,3000,0,VIC,203508942,327.94286047294736
358,72/99 spring stre...,72,99,spring,street,melbourne,3000,0,VIC,203503928,62.60909214963515
799,114/299 queen str...,114,299,queen,street,melbourne,3000,1,VIC,52559131,39.75980424561671
796,310/11 rose lane ...,310,11,rose,lane,melbourne,3000,3,VIC,422616960,57.12811692707692
341,306/181 abeckett ...,306,181,abeckett,street,melbourne,3000,3,VIC,214683784,35.720844869191446
268,304/238 flinders ...,304,238,flinders,lane,melbourne,3000,3,VIC,206306548,81.66953866486438


In [14]:
print(all_matched.count(), all_unmatched.count())

9832 1268


In [10]:
print(all_matched.count(), all_unmatched.count())

9433 1206


In [11]:
# matching unmatched entries by locality- as some listings on domain listed incorrect postcodes but correct suburb 
cond = (((all_unmatched['APT_NUM'] == all_properties['APT_NUM']) | 
        ((all_unmatched['APT_NUM'].isNull() == True) & (all_properties['APT_NUM'].isNull() == True))) &\
        (all_unmatched['BLDG_NUM'] == all_properties['BLDG_NUM']) &\
        (all_unmatched['ROAD_NAME'] == all_properties['ROAD_NAME']) &\
        (all_unmatched['ROAD_TYPE'] == all_properties['ROAD_TYPE']) &\
        (all_unmatched['LOCALITY'] == all_properties['LOCALITY']))
joined_by_locality = all_unmatched.join(all_properties, cond, "left")\
        .drop(all_properties['APT_NUM'],all_properties['BLDG_NUM'],all_properties['ROAD_NAME'],all_properties['ROAD_TYPE'],all_properties['LOCALITY'], all_properties['POSTCODE'], all_properties['APT_LVL'])
matched_by_locality = joined_by_locality.filter(F.col("area").isNotNull() == True)
not_matched_by_locality = joined_by_locality.filter(F.col("area").isNull() == True)

In [12]:
# combining entried matched by locality to previously matched data 
all_matched = all_matched.unionByName(matched_by_locality)
all_matched.write.mode('overwrite').csv('../data/curated/all_matched.csv')

In [18]:
spark.read.csv('../data/curated/real_estate_2023.csv', header=True).limit(20)

23/09/20 14:13:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, address, property_type, price_per_week, postcode, parking_spaces, bedrooms, bathrooms, furnished_found
 Schema: _c0, id, address, property_type, price_per_week, postcode, parking_spaces, bedrooms, bathrooms, furnished_found
Expected: _c0 but found: 
CSV file: file:///mnt/c/Users/clari/OneDrive/ads/real-estate-industry-project-open-source-industry-project-40-1/data/curated/real_estate_2023.csv
23/09/20 14:13:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, address, property_type, price_per_week, postcode, parking_spaces, bedrooms, bathrooms, furnished_found
 Schema: _c0, id, address, property_type, price_per_week, postcode, parking_spaces, bedrooms, bathrooms, furnished_found
Expected: _c0 but found: 
CSV file: file:///mnt/c/Users/clari/OneDrive/ads/real-estate-industry-project-open-source-industry-project-40-1/data/curated/real_estate_2023.csv


_c0,id,address,property_type,price_per_week,postcode,parking_spaces,bedrooms,bathrooms,furnished_found
0,0,904/265 Exhibitio...,Apartment / Unit ...,850.0,3000,1.0,2.0,2.0,False
1,1,210/422 Collins S...,Apartment / Unit ...,475.0,3000,null,1.0,1.0,True
2,2,1902/200 Spencer ...,Apartment / Unit ...,630.0,3000,1.0,2.0,1.0,False
3,3,312B/399 Bourke S...,Apartment / Unit ...,450.0,3000,null,1.0,1.0,False
4,4,3313/228 La Trobe...,House,900.0,3000,2.0,2.0,1.0,True
5,5,301/499 St Kilda ...,Apartment / Unit ...,470.0,3000,null,1.0,1.0,False
6,6,2302/17 Spring St...,Apartment / Unit ...,1500.0,3000,1.0,2.0,2.0,False
7,7,3201/318 Queen St...,Apartment / Unit ...,550.0,3000,null,1.0,1.0,False
8,8,1/113 brook stree...,null,600.0,3000,null,1.0,1.0,False
9,9,403/17 Singers La...,Apartment / Unit ...,450.0,3000,null,1.0,1.0,False


In [41]:
schema = StructType([StructField('index', StringType(), True),\
                    StructField('id', StringType(), True),\
                    StructField('address', StringType(), True),\
                    StructField('property_type', IntegerType(), True),\
                    StructField('price_per_week', StringType(), True),\
                    StructField('postcode', StringType(), True),\
                    StructField('parking_spaces', StringType(), True),\
                    StructField('bedrooms', StringType(), True),\
                    StructField('bathrooms', StringType(), True),\
                    StructField('furnished_found', StringType(), True),\
                    StructField('internal_area', StringType(), True),\
                    StructField('land_area', StringType(), True)])

complete_df = spark.read.csv('../data/curated/real_estate_2023.csv', header=True, schema=schema)

combined_df = all_matched['id', 'APT_LVL','LOCALITY', 'area'].join(complete_df, how="inner", on='id')

In [38]:
complete_df.limit(20)

23/09/20 14:32:36 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, address, property_type, price_per_week, postcode, parking_spaces, bedrooms, bathrooms, furnished_found, internal_area, land_area
 Schema: index, id, address, property_type, price_per_week, postcode, parking_spaces, bedrooms, bathrooms, furnished_found, internal_area, land_area
Expected: index but found: 
CSV file: file:///mnt/c/Users/clari/OneDrive/ads/real-estate-industry-project-open-source-industry-project-40-1/data/curated/real_estate_2023.csv
23/09/20 14:32:36 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, address, property_type, price_per_week, postcode, parking_spaces, bedrooms, bathrooms, furnished_found, internal_area, land_area
 Schema: index, id, address, property_type, price_per_week, postcode, parking_spaces, bedrooms, bathrooms, furnished_found, internal_area, land_area
Expected: index but found: 
CSV file: file:///mnt/c/Users/clari/OneDrive

index,id,address,property_type,price_per_week,postcode,parking_spaces,bedrooms,bathrooms,furnished_found,internal_area,land_area
0,0,904/265 Exhibitio...,null,850.0,3000,1.0,2.0,2.0,False,['92'],null
1,1,210/422 Collins S...,null,475.0,3000,null,1.0,1.0,True,['42'],null
2,2,1902/200 Spencer ...,null,630.0,3000,1.0,2.0,1.0,False,null,null
3,3,312B/399 Bourke S...,null,450.0,3000,null,1.0,1.0,False,null,null
4,4,3313/228 La Trobe...,null,900.0,3000,2.0,2.0,1.0,True,null,null
5,5,301/499 St Kilda ...,null,470.0,3000,null,1.0,1.0,False,null,null
6,6,2302/17 Spring St...,null,1500.0,3000,1.0,2.0,2.0,False,null,null
7,7,3201/318 Queen St...,null,550.0,3000,null,1.0,1.0,False,null,null
8,8,1/113 brook stree...,null,600.0,3000,null,1.0,1.0,False,null,null
9,9,403/17 Singers La...,null,450.0,3000,null,1.0,1.0,False,null,null


In [42]:
combined_df.write.mode('overwrite').csv('../data/curated/real_estate_2023_matched_area.csv')

23/09/20 14:43:28 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, address, property_type, price_per_week, postcode, parking_spaces, bedrooms, bathrooms, furnished_found, internal_area, land_area
 Schema: index, id, address, property_type, price_per_week, postcode, parking_spaces, bedrooms, bathrooms, furnished_found, internal_area, land_area
Expected: index but found: 
CSV file: file:///mnt/c/Users/clari/OneDrive/ads/real-estate-industry-project-open-source-industry-project-40-1/data/curated/real_estate_2023.csv


In [44]:
combined_df.filter((F.col('land_area').isNotNull() == True) &\
                   (F.col('APT_LVL').isNotNull() == True)).limit(20)

23/09/20 14:51:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, address, property_type, price_per_week, postcode, parking_spaces, bedrooms, bathrooms, furnished_found, internal_area, land_area
 Schema: index, id, address, property_type, price_per_week, postcode, parking_spaces, bedrooms, bathrooms, furnished_found, internal_area, land_area
Expected: index but found: 
CSV file: file:///mnt/c/Users/clari/OneDrive/ads/real-estate-industry-project-open-source-industry-project-40-1/data/curated/real_estate_2023.csv
23/09/20 14:51:41 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, address, property_type, price_per_week, postcode, parking_spaces, bedrooms, bathrooms, furnished_found, internal_area, land_area
 Schema: index, id, address, property_type, price_per_week, postcode, parking_spaces, bedrooms, bathrooms, furnished_found, internal_area, land_area
Expected: index but found: 
CSV file: file:///mnt/c/Users/clari/OneDrive

id,APT_LVL,LOCALITY,area,index,address,property_type,price_per_week,postcode,parking_spaces,bedrooms,bathrooms,furnished_found,internal_area,land_area
82,3,melbourne,57.04374857475473,82,301/68 La Trobe S...,null,650.0,3000,1.0,2.0,1.0,False,['7'],['65']
448,21,melbourne,210.7216558296796,434,2115/101 Therry S...,null,905.0,3000,null,2.0,2.0,False,null,['90']
1142,11,southbank,124.6430853609927,1056,1106/283 City Roa...,null,460.0,3006,null,1.0,1.0,False,null,['1495']
1181,24,southbank,196.2764498416121,1095,2401/45 Haig Stre...,null,675.0,3006,null,2.0,2.0,False,null,['2356']
1325,26,southbank,174.84246184848598,1236,262/183 City Road...,null,1400.0,3006,null,2.0,2.0,True,null,['100']
1165,0,southbank,778.4982074294239,1079,1/79 Dodds Street...,null,675.0,3006,1.0,2.0,1.0,False,['115'],['115']
1399,16,southbank,88.5323865265883,1306,1601/11 Bale Circ...,null,650.0,3006,null,2.0,2.0,False,null,['287326']
9670,0,ballarat north,54.607836455153304,9271,2/1010 Ligar Stre...,null,380.0,3350,2.0,3.0,2.0,False,['18'],['232']
2429,0,werribee,16.498449559601795,2289,6/42-48 Cottrell ...,null,320.0,3030,1.0,2.0,1.0,False,null,['129']
2506,0,werribee south,295.0817020075714,2364,31/656 O'Connors ...,null,650.0,3030,1.0,3.0,2.0,False,null,['262']


In [29]:
all_properties.filter((F.col("ROAD_NAME") == 'dandenong') &\
                                (F.col("ROAD_TYPE") == 'road') &\
                                (F.col("BLDG_NUM") == '1525')&\
                                (F.col("LOCALITY") == 'oakleigh') ).limit(20)

APT_LVL,BLDG_NUM,ROAD_NAME,ROAD_TYPE,LOCALITY,STATE,POSTCODE,PR_PFI,APT_NUM,area
0,1525,dandenong,road,oakleigh,VIC,3166,425187790,2,405.02012569084764
0,1525,dandenong,road,oakleigh,VIC,3166,425187789,3,405.02012569084764
0,1525,dandenong,road,oakleigh,VIC,3166,425187791,1,405.02012569084764
null,1525,dandenong,road,oakleigh,VIC,3166,424086139,null,1215.060377072543
